# Galopp-Preprocessor

For this step in the project I will use jupyter notebook, because it is way faster to visualize and more flexible on doin data preprocessing. (The exploratory data analysis part will also be with jupyter notebook)

### Imports

In [1]:
import pandas as pd
import numpy as np

### Load dataset

In [2]:
galopp = pd.read_csv("all_races.csv")
galopp.sample(5)

,Date,Location,Distance,Prize,Category,Class,Ground_state,Horses
7055,03. Oktober 2018,Hoppegarten,2800 m,6200 €,E,III,Boden: gut,"[' 1. ', ' Tamim..."
2301,21. Dezember 2014,Neuss (Sand),1500 m,0 €,NaN,NaN,Boden: nass ...,"[' 1. ', ' Call ..."
4067,22. Mai 2016,Magdeburg,1550 m,3000 €,E,NaN,Boden: gut,"[' 1. ', ' Sorel..."
197,24. April 2013,Köln,1850 m,0 €,NaN,NaN,Boden: gut,"[' 1. ', ' Raipu..."
9133,14. Oktober 2020,Mülheim,1200 m,7000 €,D,III,Boden: weich ...,"[' 1. ', ' Reine..."


### Look at general information

In [3]:
galopp.columns

Index(['Date', 'Location', 'Distance', 'Prize', 'Category', 'Class',
       'Ground_state', 'Horses'],
      dtype='object')

In [4]:
print(galopp.isna().sum())
print("")
print(galopp.isna().sum()/len(galopp)*100)

Date              15
Location          15
Distance           0
Prize              0
Category        3115
Class           4245
Ground_state       0
Horses             0
dtype: int64

Date             0.162725
Location         0.162725
Distance         0.000000
Prize            0.000000
Category        33.792580
Class           46.051204
Ground_state     0.000000
Horses           0.000000
dtype: float64


In [5]:
galopp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9218 entries, 0 to 9217
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          9203 non-null   object
 1   Location      9203 non-null   object
 2   Distance      9218 non-null   object
 3   Prize         9218 non-null   object
 4   Category      6103 non-null   object
 5   Class         4973 non-null   object
 6   Ground_state  9218 non-null   object
 7   Horses        9218 non-null   object
dtypes: object(8)
memory usage: 576.2+ KB


First thing I see is that there are some NaN values. 15 each on _Date_ and _Location_ (which are only 0.16% of the whole set). Because only a few are missing, and they are not really important to my goal, I fill them with the mode (normally not really applicable for the _Date_ , but it should be sufficient).

Also, the columns 'Category' and 'Class' are missing 3115 and 4245, which are ~33% and ~46%. Normally, because they are categorical, they could be filled with the mode, but so many values are missing, so I decide to drop them, as they are not important for my goal.

Next, the datatypes should be changed for _Distance_ and _Prize_ to int. But before that, the 'm' in _Distance_ and the '€' in _Prize_ have to be removed. (Also, there are '\xa0's in both columns for each entry! These have to be removed too before converting to int.). And, at last, fill the empty strings with a 0, otherwise this would lead to ValueErrors when converting to int.

The _Ground_state_ always contains the prefix 'Boden: ', this can also be removed.

In [6]:
# Drop columns
galopp.drop(columns=["Category", "Class"], inplace=True)

# Fill dates and location by 
galopp["Date"].fillna(galopp["Date"].mode(), inplace=True)
galopp["Location"].fillna(galopp["Location"].mode(), inplace=True)

# Remove units
galopp["Distance"] = galopp["Distance"].apply(lambda x: x.replace("m", ""))
galopp["Prize"] = galopp["Prize"].apply(lambda x: x.replace("€", ""))

# Remove bytes
galopp["Distance"] = galopp["Distance"].apply(lambda x: x.replace("\xa0", ""))
galopp["Prize"] = galopp["Prize"].apply(lambda x: x.replace("\xa0", ""))

# Fill empty strings with 0
galopp["Distance"] = galopp["Distance"].apply(lambda x: "0" if len(x) == 0 else x)
galopp["Prize"] = galopp["Prize"].apply(lambda x: "0" if len(x) == 0 else x)

# Change datatype to int
galopp["Distance"] = galopp["Distance"].astype(int)
galopp["Prize"] = galopp["Prize"].astype(int)

# Remove 'Boden: ' prefix
galopp["Ground_state"] = galopp["Ground_state"].apply(lambda x: x.replace("Boden: ", ""))

Looking at the dataframe again:

In [7]:
galopp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9218 entries, 0 to 9217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          9203 non-null   object
 1   Location      9203 non-null   object
 2   Distance      9218 non-null   int64 
 3   Prize         9218 non-null   int64 
 4   Ground_state  9218 non-null   object
 5   Horses        9218 non-null   object
dtypes: int64(2), object(4)
memory usage: 432.2+ KB


In [8]:
galopp.sample(10)

,Date,Location,Distance,Prize,Ground_state,Horses
7705,20. Juni 2019,Mannheim,1400,2500,gut,"[' 1. ', ' Mashi..."
4884,27. Dezember 2016,Dortmund (Sand),2500,6000,nass,"[' 1. ', ' Della..."
6399,21. Mai 2018,München,2000,3100,weich stellenweise gut ...,"[' 1. ', ' CMon ..."
5803,16. September 2017,Leipzig,1300,4000,gut stellenweise fest ...,"[' 1. ', ' Life ..."
7178,11. November 2018,München,2200,4000,weich stellenweise gut ...,"[' 1. ', ' Kirit..."
1469,09. Juni 2014,Hannover,2200,0,g-w,"[' 1. ', ' Schul..."
5105,01. Mai 2017,Leipzig,2000,3500,gut,"[' 1. ', ' Arabi..."
4746,21. Oktober 2016,Baden-Baden,2400,7000,weich,"[' 1. ', ' Arios..."
6172,25. März 2018,Düsseldorf,1400,10000,weich stellenweise schwer...,"[' 1. ', ' Ahraa..."
5913,15. Oktober 2017,Köln,1600,155000,gut stellenweise weich ...,"[' 1. ', ' Erasm..."


No NaNs and the fitting datatype, also the samples looking good aswell.
So for those columns everything that needs to be done is done. Lets go on with the horses per race. For this, I intend to:
- get the list of horses and make another dataframe of it
- Clean this dataset (No column names and datatypes here)
- Return the cleaned dataframe as a list and replace it
- Save each horse participation in another dataframe / csv

### Clean races and generate a participants dataframe

In [9]:
def clean_placement_string(x):
    
    if "NS" in x: # Treat "Nichtstarter", horses who didn't start the race
        x = -1
    else:
        x = x.replace(".","")
        x = x.replace("'","")
        x = x.replace("[","")
        x = x.replace("]", "")
        x = x.strip()
    
    return x

def clean_horse_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    return x

def clean_jockey_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    
    if "." in x:
        while "." in x:
            x = x[x.index(".", )+1:] # Get surname by dot
    elif len(x.split()) == 2:
        x = x.split()[1]  # Get surname when both names are in the name string
    else:
        pass
    
    return x

def clean_trainer_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    
    if "." in x:
        while "." in x:
            x = x[x.index(".", )+1:] # Get surname by dot
    elif len(x.split()) == 2:
        x = x.split()[1]  # Get surname when both names are in the name string
    else:
        x=x
    
    return x

def clean_weight_string(x):
    x = x.replace("'", "")
    x = x.replace(",",".")
    x = x.replace("]", "")
    x = x.strip()
    return x

In [10]:
# Load, clean, and replace each race
races = []
horses = []
columns = ["Place", "Horse_name", "Jockey_name", "Trainer_name", "Weight"]

for row in galopp["Horses"]:
    
    # Load row as a seperate dataset and make it a dataframe for easier editing
    split = row.split(", ")
    try:
        row_reshaped = np.array(split).reshape((-1, 5))
        race_df = pd.DataFrame(data=row_reshaped, columns=columns)

        # Clean dataset (and save a version with the races)
        race_df["Place"] = race_df["Place"].apply(clean_placement_string)
        race_df["Horse_name"] = race_df["Horse_name"].apply(clean_horse_name_string)
        race_df["Jockey_name"] = race_df["Jockey_name"].apply(clean_jockey_name_string)
        race_df["Trainer_name"] = race_df["Trainer_name"].apply(clean_trainer_name_string)
        race_df["Weight"] = race_df["Weight"].apply(clean_weight_string)

        # Add each participant to a list
        for horse in race_df.values:
            horses.append(horse)

        # Add the whole race to a list
        races.append(race_df.values)
    except:
        races.append("DELETE THIS ROW") # Some rows just dont fit... delete the rows afterwards!
    
# Save participations for further inspection
all_participations_df = pd.DataFrame(data=horses, columns=columns)
all_participations_df.to_csv("participations.csv", index=False)

In [21]:
# Replace the cleaned races with the old races in the dataframe
flattened = np.array(races).reshape(1, -1)
flattened_races = []
for dim in flattened:
    for dim2 in dim:
        try:
            flattened_races.append(dim2.reshape(1,-1))
        except:
            flattened_races.append([])
        
galopp["Horses"] = np.array(flattened_races)

In [22]:
galopp

,Date,Location,Distance,Prize,Ground_state,Horses
0,02. Dezember 2012,Neuss (Sand),1500,0,nass,"[[1, birthday prince, pietsch, , 58.0, 2, char..."
1,02. Dezember 2012,Neuss (Sand),1500,0,nass,"[[1, kreuz as, pietsch, , 58.0, 2, calyxa, sta..."
2,02. Dezember 2012,Neuss (Sand),1900,0,nass,"[[1, flashing star, seidl, , 64.0, 2, cioccomi..."
3,02. Dezember 2012,Neuss (Sand),1900,0,nass,"[[1, dreamspeed, wandt, , 56.0, 2, turgenjew, ..."
4,02. Dezember 2012,Neuss (Sand),1500,0,nass,"[[1, leoderprofi, minarik, , 53.0, 2, worlds d..."
...,...,...,...,...,...,...
9213,18. November 2020,Dresden,1900,5100,gut stellenweise weich ...,"[[1, sir vulcano, bojko, wöhler, 57.0, 2, andi..."
9214,18. November 2020,Dresden,1900,3000,gut stellenweise weich ...,"[[1, vendetta, murzabayev, dzubasz, 62.0, 2, k..."
9215,18. November 2020,Dresden,2200,3000,gut stellenweise weich ...,"[[1, koenigsstern (ire), panov, richter, 59.0,..."
9216,18. November 2020,Dresden,2200,12500,gut stellenweise weich ...,"[[1, dato (gb), pietsch, smrczek, 58.5, 2, enj..."
